<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/master/landlab_header.png"></a>


# Using the Landlab BiotaEvolver component

<hr>
<small> For instructions on how to run an interactive iPython notebook, click here: <a href="https://github.com/landlab/tutorials/blob/master/README.md">https://github.com/landlab/tutorials/blob/master/README.md</a></small><br>
<small>For more Landlab tutorials, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

In this tutorial we will:
* Introduce species into a Landlab model.
* Set the zones in which the species operate.
* Evolve a landscape and the species over time.
* Explore BiotaEvolver output data structures.

This tutorial loosely follows the fault throw experiment in Lyons et al., in preperation for Earth Surface Dynamics.

Import modules.

In [1]:
from landlab import RasterModelGrid, CLOSED_BOUNDARY, FIXED_VALUE_BOUNDARY
from landlab.components import (BiotaEvolver, FastscapeEroder, FlowRouter,
                                LinearDiffuser, NormalFault)
from landlab.components.biota_macroevolution import Species, Zone
import numpy as np

A bit of magic so that we can plot within this notebook.

In [ ]:
from __future__ import print_function
%matplotlib inline

## This

Here we will do this.

## That

Now we will do that.

## Species identifiers and phylogenetic trees

Species identifiers are automatically generated. They are stored as tuples in species objects. The first element of the identifier tuple is the clade name, and the second element is the species number. For example, the first species of clade A is 0, so the identifier is (A, 0) and the tenth species of this clade is (A, 9).

### Clade name

A clade is a group of species that share a common ancestor. Species may share multiple common ancestors, so clades can be defined differently in the same tree. The clade(s) of this example tree can be defined in multiple ways:

![](images/clade_definitions.png)

In BiotaEvolver, clades are defined by the root node for the purpose of assigning species identifiers. The species at the root node is:
* the least recent common ancestor of all species in the tree, and
* typically introduced to a BiotaEvolver model using the ```introduce_species``` function.

The clades of the first 26 species are labeled alphabetically from A to Z. The next 26 are labeled AB thru AZ. 

1-26: A to Z
27-53: AA to AZ
BA to BZ
...
AAA to AAZ
ABA to ABZ
ACA to ACZ

### Species number

Species are numbered sequently by clade beginning with 0.

In the example tree below, species A0 produced 4 child species. Only one species, B0 exists in clade B because child species were not produced by B0.

![](images/species_number.png)

### Click here for more <a href="https://github.com/landlab/landlab/wiki/Tutorials">Landlab tutorials</a>